In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import torch.utils.data.dataloader as Data
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset,DataLoader
import sys
from tqdm import tqdm

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
#Myscore function
def myScore(Target, Pred):
    A = 0
    for i in range(len(Target)):
        Er = 100 * ((Target[i] - Pred[i]) / Target[i])
        A = A + math.exp(-np.log(0.5) * (Er / 5)) if Er <=0 else A + math.exp(np.log(0.5) * (Er / 20))
    return A

In [4]:
bearing_set = 'Bearing1'

In [6]:
prediction_loss = []
# Load preprocessed data
x_train = sio.loadmat(f'./Mat_train/{bearing_set}1-4_2d_train_stride25_x.mat')['train_x']  # load sliding window preprocessed and Statistical features processed data (mean value and regression coefficient estimates feature)
y_train = sio.loadmat(f'./Mat_train/{bearing_set}1-4_2d_train_stride25_y.mat')['train_y'].transpose()

x_test = sio.loadmat('./Mat_test/Bearing1_3_test_x.mat')['test_x']
y_test = sio.loadmat('./Mat_test/Bearing1_3_test_y.mat')['test_y'].transpose()

In [7]:
print(x_train.shape)
print(y_train.shape)

(183000, 40, 2)
(183000, 1)


In [8]:
print(x_test.shape)
print(y_test.shape)

(50, 40, 2)
(50, 1)


In [9]:
x_train = Variable(torch.Tensor(x_train).float())
y_train = Variable(torch.Tensor(y_train).float())
x_test = Variable(torch.Tensor(x_test).float())
y_test = Variable(torch.Tensor(y_test).float())

In [15]:
#Hyperparameters
batch_size = 256
dim_val = 32
dim_attn = 32
dim_val_t = 32
dim_attn_t = 32
dim_val_s = 32
dim_attn_s = 32
n_heads = 4
n_decoder_layers = 1
n_encoder_layers = 2
max_rul = 125
lr = 0.001
epochs = 20
time_step = 40
dec_seq_len = 4
output_sequence_length = 1
input_size = 2
# input_size = 3

In [16]:
#Dataloader
train_dataset = TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = TensorDataset(x_test,y_test)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size = batch_size,shuffle=False)

In [17]:
# Initialize model parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# debug = True
debug = False
model = DAST(dim_val_s,dim_attn_s,dim_val_t,dim_attn_t,dim_val, dim_attn,time_step,input_size,dec_seq_len,output_sequence_length, n_decoder_layers, n_encoder_layers, n_heads, debug)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [ ]:
# print(next(model.parameters()).device)

In [18]:
#Training  and testing
loss_list = []
train_loss_list = []
test_loss_list = []
train_time = []
test_time = []
model_loss = 1000

In [19]:
for epoch in range(epochs):
    #training
    model.train()
    start1 = time.time()
    loop = tqdm(train_loader, leave=True)
    for i,(X, Y) in enumerate(loop):
        batch_X = X.to(device)
        batch_Y = Y.to(device)
        out = model(batch_X)
        # print(out[0])
        loss = torch.sqrt(criterion(out, batch_Y))
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    end1 = time.time()
    train_time.append(end1 - start1)
    loss_eopch = np.mean(np.array(loss_list))
    train_loss_list.append(loss_eopch)
    print('epoch = ',epoch,
            'train_loss = ',loss_eopch.item())

    #testing
    model.eval()
    prediction_list = []
    for j ,(batch_x,batch_y) in enumerate(test_loader):
        batch_X = batch_x.to(device)
        batch_Y = batch_y.to(device)
        start2= time.time()
        prediction = model(batch_X)
        print(prediction)
        end2 = time.time()
        test_time.append(end2 - start2)
        prediction[prediction<0] = 0
        prediction_list.append(prediction)

    out_batch_pre = torch.cat(prediction_list).detach().cpu().numpy()
    prediction_tensor = torch.from_numpy(out_batch_pre)
    test_loss = torch.sqrt(criterion(prediction_tensor, y_test))
    test_loss_list.append(test_loss.item())
    Y_test_numpy = y_test.detach().numpy()
    test_score = myScore(Y_test_numpy, out_batch_pre)
    print('test_loss = ', test_loss.item(),
            'test_score = ', test_score)

100%|██████████| 715/715 [00:32<00:00, 22.04it/s]


epoch =  0 train_loss =  1351.0332516116696
tensor([[232.9846],
        [232.9850],
        [232.9855],
        [232.9842],
        [232.9841],
        [232.9852],
        [232.9851],
        [232.9848],
        [232.9838],
        [232.9846],
        [232.9846],
        [232.9856],
        [232.9834],
        [232.9848],
        [232.9846],
        [232.9754],
        [232.9727],
        [232.9848],
        [232.9844],
        [232.9851],
        [232.9854],
        [232.9844],
        [232.9851],
        [232.9839],
        [232.9857],
        [232.9843],
        [232.9844],
        [232.9849],
        [232.9846],
        [232.9845],
        [232.9851],
        [232.9836],
        [232.9841],
        [232.9855],
        [232.9853],
        [232.9842],
        [232.9845],
        [232.9843],
        [232.9846],
        [232.9844],
        [232.9847],
        [232.9814],
        [232.9851],
        [232.9846],
        [232.9364],
        [232.9855],
        [232.9849],
        [232.983

100%|██████████| 715/715 [00:28<00:00, 25.06it/s]


epoch =  1 train_loss =  1222.5206256999836
tensor([[687.8270],
        [687.8277],
        [687.8279],
        [687.8267],
        [687.8268],
        [687.8275],
        [687.8275],
        [687.8273],
        [687.8261],
        [687.8266],
        [687.8267],
        [687.8282],
        [687.8255],
        [687.8274],
        [687.8272],
        [687.8152],
        [687.8131],
        [687.8273],
        [687.8271],
        [687.8276],
        [687.8281],
        [687.8264],
        [687.8275],
        [687.8260],
        [687.8281],
        [687.8265],
        [687.8268],
        [687.8275],
        [687.8268],
        [687.8265],
        [687.8274],
        [687.8256],
        [687.8263],
        [687.8281],
        [687.8276],
        [687.8265],
        [687.8268],
        [687.8267],
        [687.8270],
        [687.8265],
        [687.8271],
        [687.8234],
        [687.8275],
        [687.8271],
        [687.7612],
        [687.8280],
        [687.8276],
        [687.826

100%|██████████| 715/715 [00:17<00:00, 41.51it/s]


...

epoch =  17 train_loss =  871.8787510205723
tensor([[1163.7587],
        [1163.7573],
        [1163.7579],
        [1163.7581],
        [1163.7567],
        [1163.7592],
        [1163.7593],
        [1163.7572],
        [1163.7587],
        [1163.7612],
        [1163.7577],
        [1163.7573],
        [1163.7606],
        [1163.7584],
        [1163.7572],
        [1163.7551],
        [1163.7565],
        [1163.7589],
        [1163.7577],
        [1163.7581],
        [1163.7581],
        [1163.7603],
        [1163.7579],
        [1163.7600],
        [1163.7587],
        [1163.7595],
        [1163.7585],
        [1163.7560],
        [1163.7598],
        [1163.7594],
        [1163.7587],
        [1163.7603],
        [1163.7600],
        [1163.7581],
        [1163.7589],
        [1163.7593],
        [1163.7592],
        [1163.7577],
        [1163.7592],
        [1163.7599],
        [1163.7583],
        [1163.7581],
        [1163.7587],
        [1163.7570],
        [1163.7528],
        [11

100%|██████████| 715/715 [00:32<00:00, 22.02it/s]


epoch =  18 train_loss =  869.3939692593077
tensor([[1164.8795],
        [1164.8794],
        [1164.8795],
        [1164.8794],
        [1164.8790],
        [1164.8799],
        [1164.8799],
        [1164.8792],
        [1164.8796],
        [1164.8805],
        [1164.8793],
        [1164.8793],
        [1164.8802],
        [1164.8796],
        [1164.8793],
        [1164.8785],
        [1164.8788],
        [1164.8798],
        [1164.8794],
        [1164.8795],
        [1164.8795],
        [1164.8802],
        [1164.8793],
        [1164.8802],
        [1164.8796],
        [1164.8800],
        [1164.8796],
        [1164.8789],
        [1164.8800],
        [1164.8800],
        [1164.8796],
        [1164.8801],
        [1164.8800],
        [1164.8795],
        [1164.8799],
        [1164.8799],
        [1164.8799],
        [1164.8793],
        [1164.8799],
        [1164.8801],
        [1164.8796],
        [1164.8794],
        [1164.8796],
        [1164.8790],
        [1164.8776],
        [11

100%|██████████| 715/715 [00:32<00:00, 21.91it/s]

epoch =  19 train_loss =  867.1578696639054
tensor([[1163.0511],
        [1163.0508],
        [1163.0509],
        [1163.0510],
        [1163.0507],
        [1163.0511],
        [1163.0514],
        [1163.0509],
        [1163.0511],
        [1163.0518],
        [1163.0509],
        [1163.0509],
        [1163.0515],
        [1163.0511],
        [1163.0508],
        [1163.0503],
        [1163.0505],
        [1163.0513],
        [1163.0509],
        [1163.0509],
        [1163.0510],
        [1163.0516],
        [1163.0509],
        [1163.0515],
        [1163.0511],
        [1163.0515],
        [1163.0511],
        [1163.0504],
        [1163.0516],
        [1163.0514],
        [1163.0511],
        [1163.0515],
        [1163.0514],
        [1163.0509],
        [1163.0513],
        [1163.0514],
        [1163.0513],
        [1163.0509],
        [1163.0511],
        [1163.0515],
        [1163.0511],
        [1163.0510],
        [1163.0511],
        [1163.0508],
        [1163.0496],
        [11

In [19]:
#testing
model.eval()
prediction_list = []
for j ,(batch_x,batch_y) in enumerate(test_loader):
    batch_X = batch_x.to(device)
    batch_Y = batch_y.to(device)
    start2= time.time()
    prediction = model(batch_X)
    print(prediction.shape)
    end2 = time.time()
    test_time.append(end2 - start2)
    prediction[prediction<0] = 0
    prediction_list.append(prediction)

torch.Size([50, 1])


# Appendix

In [ ]:
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
test_model = test()
m = nn.Sequential(nn.Linear(2,2), 
                  nn.ReLU(),
                 nn.Sequential(nn.Sigmoid(), nn.ReLU()))
print(m.children())
print(test_model.children())

In [ ]:
print(type(prediction_list))

In [ ]:
# model.children()
# model._moduless